# Operacje wejścia i wyjścia

Operacje IO to wszelkie działania mające na celu komunikację programu z zewnętrznym światem - człowiekiem, danymi, siecią, innymi procesami itp. W tym przypadku szczególny nacisk położmy na wczytywanie i zapisywanie danych, co jest, rzecz jasna, jedną z pierwszych i podstawowych czynności w ich analizie.

---
Polecajki bibliotek:
* Serialization (do zapisu danych binarnie) - trochę jak Pickle
* SparseArrayKit
* Printf  

Do wczytywania wielkich plików co się nie mieszczą w pamięci RAM:
* JLD2
* Arrow
  
Do czytania plików `.csv`
* CSV

---

## Strumienie systemowe (stdin, stdout, stderr)
Polecenie `readline()` domyślnie wczytuje jedną linię tekstu ze strumienia `stdin`. Zamiast niego może być podany inny strumień (każdy strumień IO zawsze obsługuje conajmniej metodę read i write), np. plik, strumień sieciowy. Zawsze dostępne systemowe strumienie to `stdin`, `stdout` (domyślny argument funkcji print) oraz `stderr`.

## Strumienie systemowe (stdin, stdout, stderr)
Polecenie `readline()` domyślnie wczytuje jedną linię tekstu ze strumienia `stdin`. Zamiast niego może być podany inny strumień (każdy strumień IO zawsze obsługuje conajmniej metodę read i write), np. plik, strumień sieciowy. Zawsze dostępne systemowe strumienie to `stdin`, `stdout` (domyślny argument funkcji print) oraz `stderr`.

In [1]:
x = parse(Float64, readline(stdin))
if x < 0.0
    println(stderr, "Podaj x >= 0")
    exit(1)
end
println("√$x = ", sqrt(x))



: 

: 

### Zadanie 
Napisz program (skrypt) wczytujący 2 punkty na płaszczyźnie podane przez użytkownika (x1, y1, x2, y2), który następnie podaje równanie prostej przechodzącej przez punkty, oraz przedstawia wykres z punktami i linią. W przypadku gdy użytkownik poda błędne punkty (x1 = x2 lub dane, które nie są liczbami) program powinień wypisać odpowiednią informację w strumieniu stderr.

Następnie stwórz plik z czterami liczbami "dane.txt" i przekaż go do programu przez wywołanie

`$> cat dane.txt | julia skrypt.jl 1> wynik.txt 2> err.txt`

Wyniki powinny zostać zapisane w pliku "wynik.txt" (1> to strumień stdout), natomiast ewentualne błędy, wypisywane do strumienia `stderr` (2>) zostaną zapisane do pliku "err.txt".

## Praca z plikami

Otwarcie pliku poleceniem `open` zwraca strumień `IOStream` z którym możemy pracować podobnie jak ze strumieniami standardowymi.

In [1]:
f = open("test.txt", "w")

IOStream(<file test.txt>)

Tryby otwarcia to:

* r - (odczyt - domyślny),
* w  - zapis, skasowanie istniejącej zawartości
* a - zapis, dodawanie do istniejącej zawartości
* r+ - odczyt i zapis (plik musi istnieć)
* w+ - odczyt i zapis, skasowanie zawartości
* a+ - odczyt i zapis, dodawanie do zawartości

Do czytania służą funkcje `read`, `read!`, `readline`, `readlines`.

Do pisania `write`, `print`, `println`.

Funkcja `close` zamyka strumień.

Funkcja `eof` zwraca prawdę jeżeli strumień osiągnął koniec pliku (end-of-file).



In [2]:
write(f, "1.0\n2.0\n3.0\n")

12

In [3]:
println(f, "4.0")
close(f)

Sprawdzimy teraz zawartość pliku (open domyślnie jest w trybie r)

In [4]:
f = open("test.txt")

IOStream(<file test.txt>)

In [5]:
readline(f)

"1.0"

In [6]:
readline(f)

"2.0"

In [7]:
readline(f)

"3.0"

In [8]:
readline(f)

"4.0"

In [9]:
readline(f)

""

In [10]:
eof(f)

true

In [11]:
close(f)

Składnia `do ... end` zapewnia bezpieczne zamknięcie pliku automatycznie wywołując `close`. W poniższym przykładzie zawartość jest wczytana do wektora ciągów znaków.

In [12]:
lines = open("test.txt") do f
    readlines(f)
end

4-element Vector{String}:
 "1.0"
 "2.0"
 "3.0"
 "4.0"

To samo można osiągnąć w poniższej pętli, z różnicą taką, że dane są wczytywane linia po linii 

In [13]:
f = open("test.txt")
for line in readlines(f)
    println(line)
end
close(f)

1.0
2.0
3.0
4.0


O ile funkcje `readline/readlines` działają tekstowo, to `read` zwraca binarną tablicę bajtów. Oczywiście można ją ewentualnie przerobić na ciąg znaków

In [14]:
f = open("test.txt")
data = read(f)
println(data)
close(f)

UInt8[0x31, 0x2e, 0x30, 0x0a, 0x32, 0x2e, 0x30, 0x0a, 0x33, 0x2e, 0x30, 0x0a, 0x34, 0x2e, 0x30, 0x0a]


In [15]:
String(data)

"1.0\n2.0\n3.0\n4.0\n"

Lub od razu wczytać do odpowiedniego typu

In [16]:
f = open("test.txt")
data = read(f, String)
println(data)
close(f)

1.0
2.0
3.0
4.0



Możemy przeczytać też zadaną liczbę (maksymalną, bo może być ich mniej jeżeli osiągniemy koniec pliku) bajtów

In [17]:
f = open("test.txt")
data = read(f, 2)
close(f)
println(data)

UInt8[0x31, 0x2e]


Oraz wczytać dane do zadanej przez nas dowolnej zmiennej (wersja `read!`). Informacja zapisana w pliku w postaci binarej jest wtedy wczytywana w tylu bitach ile zawiera zmienna i reinterpretowana. W przykładzie poniżej oczywiście taka reinterpretacja bitów nie ma wiele sensu, bo zapisane dane były w postaci tekstowej i miały zupełnie inne znaczenie.

In [18]:
f = open("test.txt")
A = zeros(Int64, 2)
println(A)
read!(f, A)
println(A)
close(f)

[0, 0]
[734137531715563057, 734137540305497651]


Ale możemy stworzyć odpowiednie dane binarne i sprawdzić działanie tej metody. Stworzymy macierz losowych liczb Float64 3x2 i wczytamy ją ponownie do takiej samej macierzy wcześniej wypełnionej zerami.

In [19]:
f = open("test.bin", "w")
A = [rand(3);; randn(3)]
println(A, " ", size(A))
write(f, A)
close(f)

[0.3673362644423729 0.42516128215338655; 0.5548245707476176 -2.1808183607976686; 0.2956896669646576 0.03870551010305722] (3, 2)


In [20]:
f = open("test.bin", "r")
A = zeros(3, 2)
read!(f, A)
println(A)
close(f)

[0.3673362644423729 0.42516128215338655; 0.5548245707476176 -2.1808183607976686; 0.2956896669646576 0.03870551010305722]


Do poruszania się po strumieniu służy polecenie `skip` któremu podajemy strumień i liczbę bajtów o jaką chcemy się poruszyć (można poruszać się do przodu i tyłu). Ponieważ wpisaliśmy dane 64-bitowe, każda liczba zajmuje 8 bajtów. Idąc 8 bajtów do przodu przeskoczymy jedną liczbę, idąc 16 bajtów do tyłu cofniemy się o dwie. Aby sprawdzić czy liczby są poprawnie wczytywane trzeba pamiętać, że dane są zapisywane kolumnami (a nie rzędami).

In [21]:
f = open("test.bin", "r")
println(read(f, Float64))
println(read(f, Float64), "\n")
close(f)

f = open("test.bin", "r")
println(read(f, Float64))
skip(f, 8)
println(read(f, Float64))
skip(f, -16)
println(read(f, Float64))

close(f)

0.3673362644423729
0.5548245707476176

0.3673362644423729
0.2956896669646576
0.5548245707476176


## Pliki tekstowe

Wiele danych jest zapisywanych w postaci plików tekstowych, które pomimo iż zajmują zdecydowanie więcej miejsca (i są niepraktyczne dla dużych danych), to mają przewagę nad plikami binarnymi w postaci czytelności dla człowieka bez dodatkowych informacji o strukturze binarej i konieczności dekodowania. Standardowym formatem danych teksotwych jest CSV (coma separated values), którym zajmiemy się później, bo czytaniem w tym formacie zajmują się specjalne biblioteki. Niestety czasem dane nie stosują się do specyfikacji formatu, zawierają różne błędy, specjalne kodowania znaków itp. W takich przypadkach trzeba je wczytywać ręcznie, linijka po linijce. 

W przypadku pracy z plikami tekstowymi przydają się funkcje operujące na ciągach:
* usuwanie zadanych znaków z przodu i z tyłu: `strip`, domyślnie działa na "białe znaki" tj. takie które są identyfikowane funkcją `isspace`
* Podział ciągu na podciągi: `split` w miejscu podanych znaków lub domyślnie białych znaków
* Zastąpienie wszystkich wystąpień jakiegoś podciągu `replace`

Załóżmy, że mamy plik z danymi liczbowymi, w których zastosowano przecinek do części dziesiętnych, do oddzielenia wartości średnik, całość objęto nawiasami, na początku znajduje się tabulator, a na końcu znak końca linii.

In [22]:
s = "\t(0,01;1,2;3,5)\n"
println(s)

	(0,01;1,2;3,5)



Polecenie `strip` usunie "białe znaki" czyli w naszym przypadku tabulator i koniec linii

In [23]:
strip(s)

"(0,01;1,2;3,5)"

Aby pozbyć się nawiasów musimy podać jakie znaki chcemy usunąć z końców

In [24]:
strip(s, ['\t', '\n', ')', '('])

"0,01;1,2;3,5"

Spróbujmy podzielić teraz ciąg na podciągi

In [25]:
split(s)

1-element Vector{SubString{String}}:
 "(0,01;1,2;3,5)"

Ponieważ `split` domyślnie robi to w miejscu spacji, musimy podać średnik jako miejsce podziału ręcznie

In [26]:
split(s, ";")

3-element Vector{SubString{String}}:
 "\t(0,01"
 "1,2"
 "3,5)\n"

Jak widać najpierw będziemy musieli pozbyć się elementów z końca i początku, a potem podzielić wektor

In [27]:
split(strip(s, ['\t', '\n', ')', '(']), ";")

3-element Vector{SubString{String}}:
 "0,01"
 "1,2"
 "3,5"

Zamianę przecinków na kropki zrobimy za pomocą funkcji `replace` i możemy to zrobić na samym początku

In [28]:
replace(s, "," => ".")

"\t(0.01;1.2;3.5)\n"

In [29]:
ss = split(strip(replace(s, "," => "."), ['\t', '\n', ')', '(']), ";")

3-element Vector{SubString{String}}:
 "0.01"
 "1.2"
 "3.5"

Ostatnim elementem jest zamiana napisów na liczby

In [30]:
map(x -> parse(Float64, x), ss)

3-element Vector{Float64}:
 0.01
 1.2
 3.5

Lub, łącząc wszystko razem

In [31]:
map(x -> parse(Float64, x), split(strip(replace(s, "," => "."), ['\t', '\n', ')', '(']), ";"))

3-element Vector{Float64}:
 0.01
 1.2
 3.5

Takie operacje można oczywiście wykonywać w pętli dla każdej linii tekstu

In [32]:
data = Array{Float64, 1}()
for line in readlines(f)
    line = split(line)
    # ...
    x = parse(Float64, line[1])
    push!(data, x)
    #...
end

### Zadanie

Ze strony NBP pobrać archiwum kursów walut za rok 2021 w formacie CSV
https://www.nbp.pl/home.aspx?f=/kursy/arch_a.html

Plik jest w kodowaniu iso-8859-1, w systemach linux można sprawdzić kodowanie, a następnie dokonać konwersji do kodowania utf-8 w poniższy sposób
```
$ file -bi archiwum_tab_a_2021.csv
$ iconv -f iso-8859-1 -o utf-8 archiwum_tab_a_2021.csv archiwum.csv
```

Zadanie polega na tym, aby wczytać plik w ten sposób, aby otrzymać słownik o strukturze:

    "data" => wektor dat lub liczby dni od pierwszego dnia roku    
    "waluta" => wektor kursów danej waluty    
    np.    
    "data" => [Date("2021-01-04"), Date("2021-01-05"), Date("2021-01-07"), ...],    
    "USD" => [3.6998, 3.7031, 3.6656, ...],    
    "EUR" => [4.5485, 4.5446, 4.4973, ...],
    ...
    
Po wczytaniu danych proszę narysować wykres stosunków kilku wybranych walut do dolara (np. EUR/USD, GBP/USD, CHF/USD)

